# 🔄 Resume Training: VI → EN

Tiếp tục training từ checkpoint.

**Upload vào Kaggle Input:**
- `best_model.pt` (checkpoint)
- `tokenizer_vi.model`
- `tokenizer_en.model`
- `train.pt` (processed data)
- `val.pt` (processed data)

In [ ]:
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project
!pip install -q sentencepiece sacrebleu google-generativeai python-dotenv tqdm tensorboard pyyaml

In [ ]:
import torch
print(f'PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available(): print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"
HF_TOKEN = "YOUR_HF_TOKEN"

with open('.env', 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\nHF_TOKEN={HF_TOKEN}\n')
print('✓ .env created')

In [ ]:
# ========================================
# 👇 NHẬP ĐƯỜNG DẪN TẠI ĐÂY 👇
# ========================================

CHECKPOINT_PATH = ""      # /kaggle/input/.../best_model.pt
TOKENIZER_VI_PATH = ""    # /kaggle/input/.../tokenizer_vi.model
TOKENIZER_EN_PATH = ""    # /kaggle/input/.../tokenizer_en.model
TRAIN_PT_PATH = ""        # /kaggle/input/.../train.pt
VAL_PT_PATH = ""          # /kaggle/input/.../val.pt

In [ ]:
import os
import sys
import shutil
from pathlib import Path

# Add project to path
sys.path.insert(0, '.')

# Validate input files
for name, path in [('Checkpoint', CHECKPOINT_PATH), ('Tokenizer VI', TOKENIZER_VI_PATH), 
                   ('Tokenizer EN', TOKENIZER_EN_PATH), ('Train.pt', TRAIN_PT_PATH), 
                   ('Val.pt', VAL_PT_PATH)]:
    assert path and os.path.exists(path), f"{name} not found: {path}"
    print(f'✓ {name}: {path}')

# Load config to get correct paths
from src.config import load_config
config = load_config('config/config_vi_en.yaml')

# Get actual paths from config
checkpoint_dir = config.paths.checkpoint_dir
data_dir = config.paths.data_dir

print(f'\n📍 Config paths:')
print(f'   checkpoint_dir: {checkpoint_dir}')
print(f'   data_dir: {data_dir}')

# Copy tokenizers to config's checkpoint_dir
tok_dir = checkpoint_dir / 'tokenizers'
tok_dir.mkdir(parents=True, exist_ok=True)
shutil.copy(TOKENIZER_VI_PATH, tok_dir / 'tokenizer_vi.model')
shutil.copy(TOKENIZER_EN_PATH, tok_dir / 'tokenizer_en.model')
print(f'\n✓ Tokenizers → {tok_dir}')

# Copy processed data to config's data_dir
processed_dir = data_dir / 'processed_vi_en'
processed_dir.mkdir(parents=True, exist_ok=True)
shutil.copy(TRAIN_PT_PATH, processed_dir / 'train.pt')
shutil.copy(VAL_PT_PATH, processed_dir / 'val.pt')
print(f'✓ Processed data → {processed_dir}')

In [ ]:
# Config: use processed data
import yaml

with open('config/config_vi_en.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

cfg['data']['source'] = 'processed'

with open('config/config_vi_en.yaml', 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False)

print('✓ Config: source = processed')

In [ ]:
from src.train import Trainer

trainer = Trainer(config_path='config/config_vi_en.yaml')
trainer.setup()

print("\n" + "="*50)
print("🔄 Resuming VI → EN Training")
print("="*50)

trainer.train(resume_from=CHECKPOINT_PATH)